In [40]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
def print_short_info(df: pd.DataFrame, df_name: str, raw_file_name: str) -> None:
    print(f"Imported {df_name} from {raw_file_name}\n"
          f"with shape {df.shape}\n")
    df.info()
    return None


# Getting uncancelled bookings

First we filter `conversations_df`:
 - include rows with `booked_at`
  - exclude rows with `cancelled_at`

In [42]:
conversations_df = pd.read_csv("data/conversations_conversation.csv",
                               parse_dates=['added', 'booked_at', 'cancelled_at'])
print_short_info(conversations_df, 'conversations_df', "data/conversations_conversation.csv")

Imported conversations_df from data/conversations_conversation.csv
with shape (80180, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80180 entries, 0 to 80179
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  80180 non-null  int64         
 1   start_date          80180 non-null  object        
 2   end_date            80180 non-null  object        
 3   units               80180 non-null  int64         
 4   added               80180 non-null  datetime64[ns]
 5   booking_total       80180 non-null  int64         
 6   cancellation_fault  3373 non-null   object        
 7   requester_id        80180 non-null  int64         
 8   service_id          80180 non-null  int64         
 9   booked_at           33466 non-null  datetime64[ns]
 10  cancelled_at        3373 non-null   datetime64[ns]
dtypes: datetime64[ns](3), int64(5), object(3)
memory usage: 6.7+ MB


In [50]:
uncancelled_bookings_df = (conversations_df
                           .dropna(subset=['booked_at'], axis=0, how='any')
                           .query("cancelled_at != cancelled_at"))

In [51]:
uncancelled_bookings_df.head(40)
uncancelled_bookings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30093 entries, 5 to 80174
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  30093 non-null  int64         
 1   start_date          30093 non-null  object        
 2   end_date            30093 non-null  object        
 3   units               30093 non-null  int64         
 4   added               30093 non-null  datetime64[ns]
 5   booking_total       30093 non-null  int64         
 6   cancellation_fault  0 non-null      object        
 7   requester_id        30093 non-null  int64         
 8   service_id          30093 non-null  int64         
 9   booked_at           30093 non-null  datetime64[ns]
 10  cancelled_at        0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(5), object(3)
memory usage: 2.8+ MB


Next we join `conversations_df[['id','requester_id']]` with `conversations_review` to identify whether reviews are left by requesters
(a.k.a. owner) vs. provider.

In [52]:
reviews_df = pd.read_csv("data/conversations_review.csv")

print_short_info(reviews_df, 'reviews_df', "data/conversations_review.csv")
reviews_df.head()

Imported reviews_df from data/conversations_review.csv
with shape (28561, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28561 entries, 0 to 28560
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               28561 non-null  int64 
 1   content          28561 non-null  object
 2   stars            28561 non-null  int64 
 3   conversation_id  28561 non-null  int64 
 4   reviewer_id      28561 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.1+ MB


,id,content,stars,conversation_id,reviewer_id
0,1,Netus proin per duis dolor venenatis nam.,1,7,64386
1,2,Dolor proin donec phasellus ve suspendisse ac ...,5,9,64384
2,3,Proin ipsum urna nisl egestas justo class a ar...,5,11,64382
3,4,Porta velit lectus varius donec tellus sollici...,1,13,64381
4,5,Dolor felis.,2,15,64379


In [59]:
reviews_reviewer_df = (conversations_df[['id', 'requester_id']]
                       .rename(columns={"id": "conversation_id"})
                       .merge(reviews_df, on='conversation_id')
                       .assign(reviewed_by=lambda x: x.reviewer_id == x.requester_id)
                       .replace({"reviewed_by": {True: "Owner", False: "Provider"}})
                       )


In [60]:
reviews_reviewer_df.head()

,conversation_id,requester_id,id,content,stars,reviewer_id,reviewed_by
0,7,64386,1,Netus proin per duis dolor venenatis nam.,1,64386,Owner
1,9,64384,2,Dolor proin donec phasellus ve suspendisse ac ...,5,64384,Owner
2,11,64382,3,Proin ipsum urna nisl egestas justo class a ar...,5,64382,Owner
3,13,64381,4,Porta velit lectus varius donec tellus sollici...,1,64381,Owner
4,15,64379,5,Dolor felis.,2,64379,Owner


In [61]:
reviews_reviewer_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28561 entries, 0 to 28560
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   conversation_id  28561 non-null  int64 
 1   requester_id     28561 non-null  int64 
 2   id               28561 non-null  int64 
 3   content          28561 non-null  object
 4   stars            28561 non-null  int64 
 5   reviewer_id      28561 non-null  int64 
 6   reviewed_by      28561 non-null  object
dtypes: int64(5), object(2)
memory usage: 1.7+ MB


In [103]:
reviewer_summary_df = (reviews_reviewer_df
[['stars', 'reviewed_by']]
.groupby(['reviewed_by'], as_index=False)
.agg(['count', 'mean'])['stars']
)

In [104]:
reviewer_summary_df

,count,mean
reviewed_by,,
Owner,22499,4.415841
Provider,6062,3.904817


# Business discussion

I would plot the two metrics in a discussion with business stakeholders.

In [105]:
px.bar(reviewer_summary_df, y=['mean'])

In [106]:
px.bar(reviewer_summary_df, y=['count'])